# Hello Capstone Project Course!

## introduction:

### In this project I will analyze the severity of car accidents and try to predict the severity of a car accident using machine learning models. The data is imported from Kaggle. This notebook will be mainly used for the capstone project.

In [ ]:
import pandas as pd
import